In [1]:
import praw
import pandas as pd
import datetime as dt
from psaw import PushshiftAPI
import os


reddit = praw.Reddit(client_id='5w4Q2l4xgrT6YQ', client_secret='fs9vkUUOnmGes-NnoOEuWVSs34o', user_agent='roastme_bot')


api = PushshiftAPI(reddit)

In [8]:
import datetime as dt

start_epoch=int(dt.datetime(2015, 1, 1).timestamp())

# new_list  = list(api.search_submissions(after=start_epoch,
#                             sort_type='score',
#                             subreddit='RoastMe',
#                             filter=['url','author', 'title', 'subreddit'],
#                             limit=800))

# start_epoch=int(dt.datetime(2019, 1, 1).timestamp())

new_list  = list(api.search_submissions(after=start_epoch,
                                        sort_type='num_comments',
                                        subreddit='RoastMe'
                                        ,filter=['url','author', 'title', 'subreddit'],limit=10000))


len(new_list)


10000

In [9]:
import time
from functools import wraps

def retry(exceptions, tries=4, delay=3, backoff=2, silent=False, logger=None):
    """Retry calling the decorated function using an exponential backoff.

    :param exceptions: the exception(s) to check. may be a tuple of
        exceptions to check.
    :type exceptions: Exception type, exception instance, or tuple containing
        any number of both (eg. IOError, IOError(errno.ECOMM), (IOError,), or
        (ValueError, IOError(errno.ECOMM))
    :param tries: number of times to try (not retry) before giving up
    :type tries: int
    :param delay: initial delay between retries in seconds
    :type delay: int
    :param backoff: backoff multiplier e.g. value of 2 will double the delay
        each retry
    :type backoff: int
    :param silent: If set then no logging will be attempted
    :type silent: bool
    :param logger: logger to use. If None, print
    :type logger: logging.Logger instance
    """
    try:
        len(exceptions)
    except TypeError:
        exceptions = (exceptions,)
    all_exception_types = tuple(set(x if type(x) == type else x.__class__ for x in exceptions))
    exception_types = tuple(x for x in exceptions if type(x) == type)
    exception_instances = tuple(x for x in exceptions if type(x) != type)

    def deco_retry(f):

        @wraps(f)
        def f_retry(*args, **kwargs):
            mtries, mdelay = tries, delay
            while mtries > 1:
                try:
                    return f(*args, **kwargs)
                except all_exception_types as e:
                    if (not any(x for x in exception_types if isinstance(e, x))
                        and not any(x for x in exception_instances if type(x) == type(e) and x.args == e.args)):
                        raise
                    msg = "%s, Retrying in %d seconds..." % (str(e) if str(e) != "" else repr(e), mdelay)
                    if not silent:
                        if logger:
                            logger.warning(msg)
                        else:
                            print (msg)
                    time.sleep(mdelay)
                    mtries -= 1
                    mdelay *= backoff
            return f(*args, **kwargs)

        return f_retry  # true decorator

    return deco_retry

In [14]:

import datetime as dtstart_epoch
import cv2
import urllib
import numpy as np
# import urllib.request
import uuid 
import matplotlib.pyplot as plt
import re
from urllib.error import URLError, HTTPError
from urllib import request
from urllib.request import urlopen
import os
from praw.models import MoreComments
# from urllib.request import Request, urlopen


posts_IDs = []
new_post_ids = []
 

comments_data=open("text_data_v2.txt", "a+")

post_ID_list=open("posts_id.txt", "r")

posts_IDs = str(post_ID_list.read()).split("\n")


print("continuing from {}".format(len(posts_IDs)))

all_post_ids = posts_IDs.copy()
# print(all_post_ids)



iteration = 0
duplicates = 0

for submission in new_list:
    
    iteration = iteration + 1
    
    print("On iteration : {}".format(iteration))
        
    comment_counter = 0
    

    if submission in all_post_ids :
        
        duplicates = duplicates + 1
        
        print("Duplicates found until now : {}".format(duplicates))
                
        continue
        
    if len(submission.comments) < 20:
        
        print("Skipping less comment of {}".format(submission.id))
                
        continue
        
    

    elif str(submission.url).split(".")[-1] == "jpg":
        
        all_post_ids.append(str(submission.id))
        new_post_ids.append(str(submission.id))
        
        print(submission.url)
        
        
        try:
          
            with urllib.request.urlopen(submission.url) as url:
                arr = np.asarray(bytearray(url.read()), dtype=np.uint8)
                if arr.shape == (503,):
                    continue
    
                rgb_img = cv2.imdecode(arr, -1) 
                cv2.imwrite(os.path.join('image_data',str(submission.id)+".jpg"),rgb_img)
                

        except HTTPError as e:
            
            if e.code == 502:
                
                @retry(urllib.error.URLError, tries=4, delay=3, backoff=2)
                def urlopen_with_retry():
                    return urllib.request.urlopen(submission.url)
                urlopen_with_retry()
                
            if e.reason == 'Not Found':
                
                continue
            
            else:
                print ('Failure: ' + str(e.reason))
                
                
        for comment in submission.comments:
            
            if isinstance(comment, MoreComments):
                                
                continue
                
            if comment_counter == 10 :
                
                break
                
                
            
            a = comment.body
            
            
            if len(str(a).split("\n")) == 1 and len(a) < 90 and len(a) > 7:
                
                
                
                comment_counter = comment_counter + 1


                   
                a_filtered = " ".join(re.findall(r"[a-zA-Z0-9]+", str(a))).lower()           
                print("{}.jpg#{}    {}".format(submission.id,comment_counter,a_filtered))
                comments_data.write("{}.jpg#{}    {}\n".format(submission.id,comment_counter,a_filtered))
            
            
        
    else:
        
        continue
        
        
    


    
    


comments_data.close()
post_ID_list.close()


post_Write_file=open("posts_id.txt", "a+")

for post_id in new_post_ids:
    post_Write_file.write("{}\n".format(post_id))
    

post_Write_file.close()    









continuing from 3541
On iteration : 1
Skipping less comment of bvkgnn
On iteration : 2
https://i.redd.it/zhuyuq00pq131.jpg
bvkcf0.jpg#1    study or you ll need to start posting these with far less clothes to make a living
bvkcf0.jpg#2    the asian in me says study but the white in me craves attention
bvkcf0.jpg#3    not the first time you disappointed your parents
bvkcf0.jpg#4    with that giant noggin you are more disproportionate than a child s stick figure drawing
bvkcf0.jpg#5    this isn t right don t dox the girl who gave robert kraft a tug at that spa
On iteration : 3
Skipping less comment of bvkbaq
On iteration : 4
Skipping less comment of bvk9zn
On iteration : 5
Skipping less comment of bvk2ew
On iteration : 6
Skipping less comment of bvjyvg
On iteration : 7
Skipping less comment of bvjtib
On iteration : 8
Skipping less comment of bvjok7
On iteration : 9
Skipping less comment of bvjkyn
On iteration : 10
Skipping less comment of bvjkpz
On iteration : 11
Skipping less comment of 

Skipping less comment of bv98m4
On iteration : 159
Duplicates found until now : 41
On iteration : 160
Skipping less comment of bv975o
On iteration : 161
Skipping less comment of bv9690
On iteration : 162
Skipping less comment of bv92qn
On iteration : 163
Duplicates found until now : 42
On iteration : 164
Skipping less comment of bv8yv8
On iteration : 165
Duplicates found until now : 43
On iteration : 166
Skipping less comment of bv8vvw
On iteration : 167
Skipping less comment of bv8ucd
On iteration : 168
Skipping less comment of bv8rw1
On iteration : 169
Skipping less comment of bv8rd3
On iteration : 170
Duplicates found until now : 44
On iteration : 171
Skipping less comment of bv8mct
On iteration : 172
Duplicates found until now : 45
On iteration : 173
Skipping less comment of bv8ic9
On iteration : 174
Skipping less comment of bv8e3w
On iteration : 175
Duplicates found until now : 46
On iteration : 176
Skipping less comment of bv85vf
On iteration : 177
Skipping less comment of bv830j

Duplicates found until now : 153
On iteration : 499
Duplicates found until now : 154
On iteration : 500
Skipping less comment of buqkjp
On iteration : 501
Duplicates found until now : 155
On iteration : 502
Duplicates found until now : 156
On iteration : 503
Skipping less comment of buqd20
On iteration : 504
Skipping less comment of buqabt
On iteration : 505
Duplicates found until now : 157
On iteration : 506
Skipping less comment of buq6b6
On iteration : 507
Skipping less comment of buq59b
On iteration : 508
Duplicates found until now : 158
On iteration : 509
Duplicates found until now : 159
On iteration : 510
Skipping less comment of bupw35
On iteration : 511
Skipping less comment of bupmoo
On iteration : 512
Skipping less comment of bupkcv
On iteration : 513
Skipping less comment of bupixr
On iteration : 514
Skipping less comment of buph02
On iteration : 515
Skipping less comment of bupe1o
On iteration : 516
Skipping less comment of bupcir
On iteration : 517
Skipping less comment of

On iteration : 760
Duplicates found until now : 244
On iteration : 761
Skipping less comment of bug4ue
On iteration : 762
Skipping less comment of bug4cu
On iteration : 763
Skipping less comment of bug3qe
On iteration : 764
Duplicates found until now : 245
On iteration : 765
Skipping less comment of bufz1s
On iteration : 766
Skipping less comment of bufv38
On iteration : 767
Skipping less comment of bufsk4
On iteration : 768
Skipping less comment of bufre7
On iteration : 769
Duplicates found until now : 246
On iteration : 770
Skipping less comment of bufo29
On iteration : 771
Skipping less comment of bufngp
On iteration : 772
Skipping less comment of bufksp
On iteration : 773
Skipping less comment of bufjkg
On iteration : 774
Skipping less comment of bufgsz
On iteration : 775
Skipping less comment of bufgs1
On iteration : 776
Skipping less comment of bufga3
On iteration : 777
Skipping less comment of bufdf4
On iteration : 778
Skipping less comment of bufcuz
On iteration : 779
Skipping 

Skipping less comment of bu11kd
On iteration : 1030
Skipping less comment of bu0uuy
On iteration : 1031
Duplicates found until now : 331
On iteration : 1032
Skipping less comment of bu0sdv
On iteration : 1033
On iteration : 1034
Skipping less comment of bu0nf3
On iteration : 1035
Skipping less comment of bu0btz
On iteration : 1036
Skipping less comment of bu0a5q
On iteration : 1037
Skipping less comment of bu09eh
On iteration : 1038
Skipping less comment of bu08n3
On iteration : 1039
Duplicates found until now : 332
On iteration : 1040
Skipping less comment of btzssp
On iteration : 1041
Skipping less comment of btznwj
On iteration : 1042
Duplicates found until now : 333
On iteration : 1043
Skipping less comment of btzjk1
On iteration : 1044
Skipping less comment of btzj1r
On iteration : 1045
Skipping less comment of btzcwj
On iteration : 1046
Skipping less comment of btz8jp
On iteration : 1047
Skipping less comment of btz2ho
On iteration : 1048
Skipping less comment of btz274
On iterat

...

Skipping less comment of bglu4p
On iteration : 9699
Duplicates found until now : 1765
On iteration : 9700
Skipping less comment of bglrna
On iteration : 9701
Skipping less comment of bglnc9
On iteration : 9702
Skipping less comment of bgllie
On iteration : 9703
Skipping less comment of bgllbs
On iteration : 9704
Skipping less comment of bgli4n
On iteration : 9705
Skipping less comment of bglfs0
On iteration : 9706
Skipping less comment of bgleqf
On iteration : 9707
Skipping less comment of bglc6v
On iteration : 9708
https://i.redd.it/6beo0uwmq2u21.jpg
bglbcw.jpg#1    you look like you use a harley davidson exhaust at full rev as a dildo
bglbcw.jpg#2    deleted
bglbcw.jpg#3    not giving a fuck about your appearance isnt anarchy
bglbcw.jpg#4    you re a lesbian anarchist so what s your war cry nagging
bglbcw.jpg#5    sublime shirt eh so you just started smoking pot what 8 10 months ago
On iteration : 9709
Skipping less comment of bgla83
On iteration : 9710
Skipping less comment of bgl9z

bgis2w.jpg#1    my great grandma had that same shirt
bgis2w.jpg#2    you ve definitely been shafted more than that dart board
bgis2w.jpg#3    you look like you re still homeless
bgis2w.jpg#4    you misspelled mangina
bgis2w.jpg#5    about to be homeless again
On iteration : 9774
Skipping less comment of bgirn4
On iteration : 9775
https://i.redd.it/lywbvxi0o1u21.jpg
bgirhd.jpg#1    did he just say the n word without a pass
bgirhd.jpg#2    jew unit
bgirhd.jpg#3    you just look like you have a lisp
bgirhd.jpg#4    omg it s corey matthews
bgirhd.jpg#5    i m white so i can t say that but whatever you are it is not that
On iteration : 9776
https://i.redd.it/mtsjjkx0n1u21.jpg
On iteration : 9777
Skipping less comment of bgimdl
On iteration : 9778
Skipping less comment of bgilhc
On iteration : 9779
On iteration : 9780
Skipping less comment of bgihw3
On iteration : 9781
https://i.redd.it/6kt6tunnj1u21.jpg
bgigpy.jpg#1    a bow tie on a teachers salary those 12 monthly payments of 1 99 must be

Skipping less comment of bgdx36
On iteration : 9859
Skipping less comment of bgdshk
On iteration : 9860
Skipping less comment of bgdr4n
On iteration : 9861
Skipping less comment of bgdou3
On iteration : 9862
Skipping less comment of bgdol4
On iteration : 9863
Skipping less comment of bgdohy
On iteration : 9864
https://i.redd.it/c1pn3n4iyyt21.jpg
On iteration : 9865
https://i.redd.it/fctnhz4vxyt21.jpg
bgdmo7.jpg#1    i think you mean r roathtme
bgdmo7.jpg#2    trailer park vampire count methula
bgdmo7.jpg#3    her work is marked by two distinct puncture wounds left on each donut
bgdmo7.jpg#4    please show your teeth i want to know how meth damage teeth
bgdmo7.jpg#5    this is where gangrel fucked off to after the wwe
On iteration : 9866
https://i.redd.it/wl2hazp4ryt21.jpg
bgdcbg.jpg#1    he s got spock s ears and e t s finger do europe a favour and keep him there
bgdcbg.jpg#2    felt cute might annex poland later idk
bgdcbg.jpg#3    you re a mean one mr grinch
bgdcbg.jpg#4    ivan drag

Skipping less comment of bganug
On iteration : 9916
Skipping less comment of bgamm9
On iteration : 9917
Skipping less comment of bgamh1
On iteration : 9918
Skipping less comment of bgalcm
On iteration : 9919
Skipping less comment of bgak1d
On iteration : 9920
https://i.redd.it/czzcdykc7xt21.jpg
bgaj6b.jpg#1    you look like a half melted claymation mccauley culkin homo alone
bgaj6b.jpg#2    you look like an alt right pez dispenser
bgaj6b.jpg#3    you got a little something on your wrist still
bgaj6b.jpg#4    i have more muscle in my left butt cheek then you do in your entire toothpick body
bgaj6b.jpg#5    dorian from the mask s non binary son
On iteration : 9921
Skipping less comment of bgagwu
On iteration : 9922
Skipping less comment of bgagg4
On iteration : 9923
Skipping less comment of bgafs3
On iteration : 9924
https://i.redd.it/4zg5z6as4xt21.jpg
bgadxp.jpg#1    a big mouth character in real life
bgadxp.jpg#2    you look like the retarded brother of kid rock
bgadxp.jpg#3    sean pe

https://i.redd.it/lh27gcoahvt21.jpg
On iteration : 9999
Skipping less comment of bg6o5t
On iteration : 10000
Skipping less comment of bg6mpt


In [ ]:
from PIL import Image


i = 0
for image in os.listdir("image_data_1"):
    size = int(os.path.getsize(os.path.join("image_data_1",image)))
#     print(size)
    
    if size == 0:
        i = i + 1
        
        os.system("rm image_data_1/{}".format(image))

    
       
        




In [24]:
with urllib.request.urlopen('http://i.imgur.com/FcyzQLf.jpg') as url:
    
    test = np.asarray(bytearray(url.read()), dtype=np.uint8)
    
test.shape    

(503,)

In [ ]:
f = open("roastme_test_data_final.txt", "r")
data = str(f.read())
all_lines = data.split("\n")

all_comment_ids = []

for each_line in all_lines:
    name = str(each_line).split("     ")[0].split(".")[0]
    all_comment_ids.append(name)
    
    
print("Total images names in comments including duplicates : "+str(len(all_comment_ids)))    
 


    

In [ ]:
filtered = list(dict.fromkeys(all_comment_ids))
print("Total images names in comments : "+str(len(filtered)))

len(filtered)


In [ ]:
import os

image_names = []

for image_name in os.listdir("final_images"):
    name = str(image_name).split(".")[0]
    image_names.append(name)
    

print("Total images : "+str(len(image_names)))

len(image_names)

In [ ]:
to_remove = [item for item in image_names if item not in filtered]
len(to_remove)

In [ ]:
# Sanity check for duplication in images dir

unique_names = list(dict.fromkeys(image_names))

len(unique_names)

In [ ]:
# counter = 0

# for img_name in filtered:
#     for image in os.listdir("224_224_image"):
#         name = str(image).split(".")[0]
#         if str(img_name) == str(name):
#             os.system("cp {}".format(os.path.join("final_image/",image)))
#             counter = counter + 1
#             print("Total copied image : {}".format(counter))
#         else:
#             continue
            
   



In [ ]:
# for name in filtered:
#     print(name)
    

In [ ]:
def disjoint(e,f):
    c = e.copy() 
    d = f.copy()
    for i in e: # no need for index. just walk each items in the array
        for j in f:
            if i == j: # if there is a match, remove the match.
                c.remove(i)
                d.remove(j)
    return c + d

matched = disjoint(image_names,filtered)

len(matched)

In [ ]:


f = open("roastme_text_data.txt", "r")
data = str(f.read())
all_lines = data.split("\n")


   

In [ ]:
match = 0 

final=open("roastme_test_data_final.txt", "a+")


for c in all_lines:
    for i in matched:
        if c.split("     ")[0].split(".")[0] == i:
            match = match+1
            print("Matched [{}] => [{}]".format(c.split("     ")[0].split(".")[0],i))
            final.write("{}\n".format(c))
            
            
print(match)
final.close()


In [ ]:
import os



for img in os.listdir("224_224_image"):
    for i in matched:
        if img.split('.')[0] == i:
            match = match+1
            os.system("cp 224_224_image/{} final_images/{}".format(img,img))
            
            
            



In [ ]:
# Final sanity check
f = open("roastme_test_data_final.txt", "r")
data = str(f.read())
all_lines = data.split("\n")




In [ ]:
'c0c61eaa-7f72-11e9-8062-3ca0674b5b67' in matched